<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>7. Proyecto de regresión</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Topología de linea base](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de la toplogía](#section3)
    * [3.1. Evaluar una topología más profunda](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, trabajaremos cómo desarrollar y evaluar modelos de redes neuronales para un problema de regresión:
* Cómo crear un modelo de red neuronal para un problema de regresión.
* Cómo utilizar scikit-learn con Keras para evaluar modelos mediante validación cruzada.
* Cómo realizar la preparación de datos para mejorar la habilidad.
* Cómo Optimizar la topología de red de modelos.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Topología de linea base</font>

En esta sección crearemos un modelo de red neuronal de referencia para el problema de regresión. Comencemos importando todas las funciones y objetos que necesitaremos para este tutorial.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Boston House Price](http://lib.stat.cmu.edu/datasets/boston)

In [11]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# load dataset
r = "C:/Users/Maria Luisa/OneDrive/Documentos/Cursos/DeepLearningConPythonyKerasRedesNeuronalesAvanzado/ProyectoClassificacionBinario/Datasets/housing.csv"

df = pd.read_csv(r, delim_whitespace=True, header=None)
dataset = df.values
# split into input (X) and output (Y) variables
X = dataset[:, 0:13]
y = dataset[:,13]

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Tiene una sola capa oculta completamente conectada con el mismo número de neuronas que los atributos de entrada (13). 
2. La red utiliza la función de activación ReLu para la capa oculta.
    * Al ser problema de regresión no tiene función de activación en la capa de salida
3. Utiliza el algoritmo de optimización ADAM y se optimiza una función de pérdida de error cuadrático medio. 
4. El objeto Wrapper para regresión se llama `KerasRegressor`. 
5. Evaluamos este modelo de línea de base con 10-fold.

In [12]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim =13, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimator = KerasRegressor(build_fn = baseline_model, epochs=100, batch_size = 5,
                          verbose=0)
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('Linea base: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Linea base: -45.76 (24.74)MSE 


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Podemos utilizar el marco de trabajo [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) para realizar la **estandarización** durante el proceso de evaluación del modelo, dentro de cada fold. 

In [13]:
# Libraries
from sklearn .preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# evaluate model with standardized dataset
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('NN', KerasRegressor(build_fn=baseline_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('Estandarizado: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Estandarizado: -52.04 (37.01)MSE 


<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
**Trabajo a realizar:** Una extensión adicional de esta sección sería aplicar de manera similar un cambio de escala a la variable de salida, como normalizarla al rango de 0 a 1 y usar una función de activación Sigmoide o similar en la capa de salida para reducir las predicciones de salida al mismo rango.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de la toplogía</font>

En esta sección evaluaremos dos topologías (más profunda y amplia) para mejorar aún más el rendimiento del modelo.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más profunda</font>

Una forma de mejorar el rendimiento de una red neuronal es agregar más capas para hacerla más profunda. En este caso con aproximadamente la mitad del número de neuronas. Nuestra topología de red ahora se ve así:
```
    13 entradas -> [13 -> 6] -> 1 salida
```
Podemos evaluar esta topología de red de la misma manera que anteriormente, al mismo tiempo que usamos la estandarización que demostró un mejor rendimiento.

In [14]:
# define the model
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim =13, activation = 'relu'))
    model.add(Dense(6, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('DeepNN', KerasRegressor(build_fn=larger_model, epochs=100,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('DeepNN: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

DeepNN: -50.73 (40.42)MSE 


<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Otro enfoque para aumentar la capacidad de representación del modelo es crear una red más amplia. Aquí, hemos aumentado el número de neuronas en la capa oculta en de 13 a 20. La topología de nuestra red más amplia se puede resumir de la siguiente manera:

```
    13 entradas -> [20] -> 1 salida
```

In [17]:
# define wider model
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim =13, activation = 'relu'))
    model.add(Dense(1))
    
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# evaluate model
estimators = []
estimators.append(('Standarize', StandardScaler()))
estimators.append(('Deep wider NN', KerasRegressor(build_fn=wider_model, epochs=50,
                                       batch_size=5, verbose=0)))
kfold = KFold(n_splits =10)
results = cross_val_score(estimator, X, y, cv= kfold)
print('Deep wider NN: %.2f (%.2f)MSE ' %(results.mean(), results.std()))

Deep wider NN: -39.18 (28.10)MSE 


La construcción del modelo ve una caída adicional en el error a aproximadamente 22 mil dólares cuadrados. Este no es un mal resultado para este problema.

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>